[View in Colaboratory](https://colab.research.google.com/github/GeniGaus/100DaysOfMLCode/blob/master/BoltzmannMachine_MovieRecommendation.ipynb)

# Restricted Boltzmann Machine(RBM)
------------------

### RBM as Energy-based model

RBM can be viewed as energy based model where energy of the system is defined by the weights of the nodes. In order to optimize the RBM, we need to minimize its energy by finding out the gradient.


### RBM as graphical model

RBM can also be viewed as a probabilistic graphical model where to optimize it, we maximize the log-likelihood of the training set.

--------------------

In order to minimize energy or maximize the log-likelihood, we compute the gradients. Since the computation of these gradients is very heavy, we use gradient approximation techniques to find the gradient and make small adjustments in the direction of the right gradient.

This is slightly similar to the process followed by supervised learning models. There we try to minimize the loss function. For that we compute the gradient of the loss function and updated the weights in the direction of minimum loss.

Similarly, for RBM, we compute the gradient and update the weights. Since the direct computation of gradients is too heavy, we approximate these gradients. For that we use Contrastive Divergence.

### Contrastive Divergence (CD)

**CD** is an algorithm to approximate the gradients. It uses **Gibbs Sampling**. **Gibbs Sampling** consists of creating the *Gibbs chain* in k steps. *Gibbs chain* consist of sampling k times the visible nodes and hidden nodes. For this, we take the input vector $v_0$ and then based on probabilities $p(h|v)$, we sample hidden nodes. Then we take the sampled hidden nodes vector as input and using $p(v|h)$, we sample the input vector $v_1$.

We reapeat this process **k times** and this is called as **$CD_k$**.

![k-step CD Algo](https://raw.githubusercontent.com/GeniGaus/100DaysOfMLCode/master/assets/CDk.png)



In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22278 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.0-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...


In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

In [5]:
## Install Pytorch

# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision

tcmalloc: large alloc 1073750016 bytes == 0x5c05a000 @  0x7fb87601c2a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [0]:
import os
os.chdir('drive/BoltzmannMachine')

In [10]:
!ls -l

total 447
-rw-r--r-- 1 root root 448626 Oct 15 09:16 AItRBM-proof.pdf
drwxr-xr-x 2 root root   4096 Oct  7 13:16 ml-100k
drwxr-xr-x 2 root root   4096 Oct  7 13:16 ml-1m


### Building the model using Pytorch

------------------


In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable


In [0]:
# importing dataset
movies = pd.read_csv('ml-1m/movies.dat', sep='::', engine='python', encoding='latin-1')
users = pd.read_csv('ml-1m/users.dat', sep='::', engine='python', encoding='latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', engine='python', encoding='latin-1')

In [0]:
# forming training and test set
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')
training_set = np.array(training_set, dtype='int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype='int')

In [0]:
# total no. of users and movies
nb_users = max(max(training_set[:, 0]), max(test_set[:, 0]))
nb_movies = max(max(training_set[:, 1]), max(test_set[:, 1]))

In [0]:
# Create training and test matrices where each line is user, each column is movie and each cell contains the rating which that user gave for the movie.
# If the user gave no rating, then 0 is placed in that cell.
# This will create the input structure which the Boltzmann machine expects,i.e. observations in lines and features in columns.

'''rating_matrix = np.zeros((nb_users, nb_movies), dtype='int')
full_set = np.concatenate((training_set, test_set), axis=0)
for i in range(len(full_set)):
  rating_matrix[full_set[i][0]][full_set[i][1]] = full_set[i][2]
'''

def convert(data):
  converted_data = []
  for user_id in range(nb_users + 1):
    movies_ids = data[:, 1][data[:, 0] == user_id]
    ratings_data = data[:, 2][data[:, 0] == user_id]
    ratings = np.zeros((nb_movies))
    ratings[movies_ids - 1] = ratings_data
    converted_data.append(ratings)
  
  return converted_data
 
training_set = convert(training_set)
test_set = convert(test_set)

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [0]:
### Converting ratings into binary ratings 0(not liked) or 1(liked). 

training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

### RBM
-----------

Create RBM from scratch

In [0]:
class RBM():
  
  def __init__(self, nv, nh):
    self.W = torch.randn((nh, nv))
    self.bh = torch.randn((1, nh))
    self.bv = torch.randn((1, nv))
  
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.bh.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.bv.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  
  def train(self, v0, vk, ph0, phk):
    self.W += torch.mm(ph0, v0) - torch.mm(phk, vk)
    self.bv += torch.sum(v0 - vk, 0)
    self.bh += torch.sum(ph0 - phk, 0)

In [0]:
nb_epochs = 10
batch_size = 100
k_steps = 10

nv = len(training_set[0])
nh = 100
rbm = RBM(nv, nh)

In [41]:
# training the RBM model
for epoch in range(1, nb_epochs + 1):
  training_loss = 0
  counter = 0.  # for normalization of loss
  for id_user in range(0, nb_users - batch_size, batch_size):
    v0 = training_set[id_user:id_user+batch_size]
    vk = training_set[id_user:id_user+batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(k_steps):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0,vk,ph0,phk)
    training_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
    counter += 1.
  print('epoch: '+str(epoch)+' loss: '+str(training_loss/counter))

epoch: 1 loss: tensor(0.3129)
epoch: 2 loss: tensor(0.2519)
epoch: 3 loss: tensor(0.2550)
epoch: 4 loss: tensor(0.2491)
epoch: 5 loss: tensor(0.2525)
epoch: 6 loss: tensor(0.2517)
epoch: 7 loss: tensor(0.2452)
epoch: 8 loss: tensor(0.2447)
epoch: 9 loss: tensor(0.2417)
epoch: 10 loss: tensor(0.2496)


In [48]:
# testing the RBM model
test_loss = 0
counter = 0.
for id_user in range(nb_users):
  v = training_set[id_user:id_user+1]
  vt = test_set[id_user:id_user+1]
  if len(v[vt >= 0]) > 0:
    _,h = rbm.sample_h(v)
    _,v = rbm.sample_v(h)
    test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
    counter += 1
print('test loss: '+str(test_loss/counter))

test loss: tensor(0.2290)
